In [2]:
:dep graph-ds = { path = "." }
:dep h3o = "0.3.0"
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "histogram", "bitmap_encoder"] }

In [3]:
use graph_ds::{Graph, Node, Edge};
use graph_ds::hexagon_graph::{h3_network_from_gtfs, h3_network_from_osm, osm::OSMLayer};
use h3o::Resolution;
use std::time::Instant;

extern crate plotters;
use plotters::prelude::*;

In [4]:
let start = Instant::now();
let mut osm_graph = h3_network_from_osm("../resources/copenhagen-with-ways.osm.pbf", OSMLayer::Walking).unwrap();
println!("osm graph created with {} nodes in {} s", osm_graph.nr_nodes(), start.elapsed().as_secs());

// let start = Instant::now();
// let mut gtfs_graph = h3_network_from_gtfs("../resources/rejseplanen.zip").unwrap();
// println!("gtfs graph created with {} nodes in {} s", gtfs_graph.nr_nodes(), start.elapsed().as_secs());

// let start = Instant::now();
// osm_graph.merge(&mut gtfs_graph)?;

// println!("merged graph created with {} nodes in {} s", osm_graph.nr_nodes(), start.elapsed().as_secs());

processing osm pbf file: ../resources/copenhagen-with-ways.osm.pbf
converted OSM file into 446019 edges
osm graph created with 364511 nodes in 2 s


In [ ]:
// let (WIDTH, HEIGHT) = (1024, 1024);

// evcxr_figure((WIDTH, HEIGHT), |root| {
//     let mut chart = ChartBuilder::on(&root)
//         .margin(10)
//         .x_label_area_size(0)
//         .y_label_area_size(0)
//         .build_cartesian_3d(53.0_f32 .. 57.0_f32, 10.0_f32 .. 14.0_f32, -2.0_f32 .. 2.0_f32)?;

//     chart.with_projection(|mut pb| {
//         pb.pitch = 0.2;
//         pb.yaw = 0.2;
//         pb.scale = 1.0;
//         pb.into_matrix()
//     });	
        
//     chart.draw_series(
//         plot_data.into_iter().map(|data| {
//             PathElement::new(vec![data.0, data.1], RED.mix(0.5))
//         })
//     )?;

//     Ok(())
// })

In [5]:
fn plot_png(plot_data: Vec<((f32, f32, f32), (f32, f32, f32))>) -> Result<(), Box<dyn std::error::Error>> {
    println!("plotting {} edges, e.g. {:?}", plot_data.len(), plot_data.get(0));

    let root = plotters::backend::BitMapBackend::new("test.png", (1024, 1024)).into_drawing_area();

    let mut chart = ChartBuilder::on(&root)
        .margin(10)
        .x_label_area_size(0)
        .y_label_area_size(0)
        .build_cartesian_3d(53.0_f32 .. 57.0_f32, 10.0_f32 .. 14.0_f32, -2.0_f32 .. 2.0_f32)?;

    chart.with_projection(|mut pb| {
        pb.pitch = 0.2;
        pb.yaw = 0.2;
        pb.scale = 1.0;
        pb.into_matrix()
    });	
        
    chart.draw_series(
        plot_data.into_iter().map(|data| {
            PathElement::new(vec![data.0, data.1], RED.mix(0.5))
        })
    )?;

    root.present()?;
    Ok(())
}

plot_png(osm_graph.get_plot_data().unwrap())

Error: failed to resolve: could not find `BitMapBackend` in `backend`